In [1]:
import numpy
from helper import *
from matplotlib import pyplot
%matplotlib inline

In [2]:
nx , ny = 101, 101
xmin, xmax = 0.0, 1.0
ymin, ymax = -0.5, 0.5
Lx, Ly = xmax-xmin, ymax-ymin
dx, dy = Lx/(nx-1), Ly/(ny-1)

x, y = numpy.linspace(xmin, xmax, num=nx), numpy.linspace(ymin, ymax, num=ny)
X, Y = numpy.meshgrid(x, y)

b = (-2.0 * (numpy.pi / 2)**2 *
     numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly))

p0 = numpy.zeros((ny, nx))
p_exact = poisson_solution(x, y, Lx, Ly)

In [3]:
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        return (-4.0 * p[1:-1, 1:-1] + 
                 p[1:-1, :-2] + p[1:-1,2:] + 
                 p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p)
    Ar = numpy.zeros_like(p)
    conv = []
    diff = rtol + 1
    ite = 0
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
        Ar[1:-1, 1:-1] = A(r)
        alpha = numpy.sum(r * r) / numpy.sum(r * Ar)
        p = pk + alpha * r
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv   

In [4]:
p, ites, conv_sd = poisson_2d_steepest_descent(p0, b, dx, dy, maxiter = 20000, rtol=1e-10)
print('Method of steepest descent: {} interetations'.format(ites)+'to reach a relative difference of {}'.format(conv_sd[-1]))

Method of steepest descent: 2 interetationsto reach a relative difference of 1.3307695446303778e-16


In [5]:
l2_norm(p, p_exact)

0.7499794373094477

In [6]:
def poisson_2d_conjugate_gradient(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    def A(p):
        return (-4.0 * p[1:-1,1:-1] + 
               p[1:-1, :-2] + p[1:-1, 2:] +
               p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p)
    Ad = numpy.zeros_like(p)
    conv = []
    diff = rtol + 1
    ite = 0
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
    d = r.copy()
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        rk = r.copy()
        Ad[1:-1, 1:-1] = A(d)
        alpha = numpy.sum(r * r) / numpy.sum(d * Ad)
        p = pk + alpha * d
        r = rk - alpha * Ad
        beta = numpy.sum(r*r)/numpy.sum(rk * rk)
        d = r + beta * d
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [7]:
p, ites, conv_cg = poisson_2d_conjugate_gradient(p0, b, dx, dy, maxiter=20000, rtol=1e-10)
print('Method of conjugate gradients: {} iterations '.format(ites)
      + 'to reach a relative difference of {}'.format(conv_cg[-1]))

Method of conjugate gradients: 2 iterations to reach a relative difference of 1.2982770796281907e-16


In [8]:
l2_norm(p, p_exact)

0.7499794373094477

In [9]:
p, ites, conv_jacobi = poisson_2d_jacobi(p0, b, dx, dy,maxiter=40000,rtol=1e-10)
print('Jacobi relaxation: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_jacobi[-1]))

Jacobi relaxation: 31227 iterations to reach a relative difference of 9.997923503623598e-11


In [10]:
b = (numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly) + numpy.sin(6.0 * numpy.pi * X / Lx) * numpy.sin(6.0 * numpy.pi * Y / Ly))

In [12]:
maxiter, rtol = 40000, 1e-10
p, ites, conv = poisson_2d_jacobi(p0, b, dx, dy, maxiter=maxiter, rtol=rtol)
print('Jacobi: {} iterations'.format(ites))
p, ites, conv = poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=maxiter, rtol=rtol)
print('steepest descent: {} iterations'.format(ites))
p, ites, conv = poisson_2d_conjugate_gradient(p0, b, dx, dy,maxiter=maxiter, rtol=rtol)
print('conjugate gradients: {} iterations'.format(ites))

Jacobi: 31226 iterations
steepest descent: 27059 iterations
conjugate gradients: 3 iterations
